<a href="https://colab.research.google.com/github/pierobaffa/ChatbotForEducation/blob/main/ChatbotForEducation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense  
from random import choice

In [ ]:
with open("/intents.json") as model_file:
    data = json.load(model_file)

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
dictionary = set({})
intents = []
patterns = []

In [ ]:
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        pattern = pattern.lower()
        tokens = nlp(pattern)
        doc = ""
        for token in tokens:
            if(not token.is_punct and not token.is_stop):
                doc+=" "+token.lemma_
                dictionary.add(token.lemma_)
        if(len(doc)>0):
            patterns.append(doc.strip())
            intents.append(intent["tag"])

In [ ]:
bow = CountVectorizer() 
X = bow.fit_transform(patterns) 
X = X.toarray()

le = LabelEncoder()
y = le.fit_transform(intents)

ohe = OneHotEncoder()
y = ohe.fit_transform(y.reshape(-1, 1)) 
y = y.toarray()

X, y = shuffle(X, y, random_state=0)

In [ ]:
model = Sequential()
model.add(Dense(12, activation="relu", input_dim=X.shape[1]))
model.add(Dense(8, activation="relu"))
model.add(Dense(y.shape[1], activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [ ]:
model.fit(X, y, epochs=500)

Epoch 1/500
5/5 [==============================] - 1s 3ms/step - loss: 3.5611 - accuracy: 0.0074
Epoch 2/500
5/5 [==============================] - 0s 4ms/step - loss: 3.5549 - accuracy: 0.0074
Epoch 3/500
5/5 [==============================] - 0s 3ms/step - loss: 3.5497 - accuracy: 0.0074
Epoch 4/500
5/5 [==============================] - 0s 3ms/step - loss: 3.5450 - accuracy: 0.0148
Epoch 5/500
5/5 [==============================] - 0s 3ms/step - loss: 3.5405 - accuracy: 0.0519
Epoch 6/500
5/5 [==============================] - 0s 3ms/step - loss: 3.5360 - accuracy: 0.0593
Epoch 7/500
5/5 [==============================] - 0s 3ms/step - loss: 3.5317 - accuracy: 0.0963
Epoch 8/500
5/5 [==============================] - 0s 3ms/step - loss: 3.5276 - accuracy: 0.0963
Epoch 9/500
5/5 [==============================] - 0s 3ms/step - loss: 3.5231 - accuracy: 0.0963
Epoch 10/500
5/5 [==============================] - 0s 3ms/step - loss: 3.5185 - accuracy: 0.1037
Epoch 11/500
5/5 [===========

In [ ]:
def preprocess(sentence):
    tokens = nlp(sentence.lower())
    doc = ""
    for token in tokens:
        if(not token.is_punct and not token.is_stop):
            doc+=" "+token.lemma_ 
    x = bow.transform([doc])
    return x



def get_response(intent_name):
    for intent in data["intents"]:
      if(intent["tag"]==intent_name):
        return choice(intent["responses"])
    
def chatbot(sentence):
    x = preprocess(sentence)
    y_proba = model.predict(x)[0]
    if(y_proba.max()>.7):
        y = y_proba.argmax()
        intent = le.inverse_transform([y])
        return get_response(intent)
    else:
      return "Sorry, I don't understand"
  

sentence = ""
while(sentence.lower()!="goodbye"):
  sentence = input("You: ")
  response = chatbot(sentence)
  print("Chatbot: "+response) 

You: hi
1/1 [==============================] - 0s 8ms/step
Chatbot: Good to see you again
You: are you an AI?
1/1 [==============================] - 0s 7ms/step
Chatbot: I am PyBot, a virtual assistant
You: how can you help me
1/1 [==============================] - 0s 7ms/step
Chatbot: I am an artificial intelligence developed to teach programming in Python
You: what is Python 
1/1 [==============================] - 0s 7ms/step
Chatbot: Python is a high-level programming language, first released publicly in 1991 by its creator Guido van Rossum. It derives its name from the comedy Monty Python's Flying Circus by the famous Monty Python.
You: what is a dictionary 
1/1 [==============================] - 0s 7ms/step
Chatbot: Dictionaries (dict) are a built-in, mutable, and unsorted type that contains items consisting of a key and a value. Once the dictionary is created and valued with a set of <key, value> pairs, the key (which must be unique) can be used to obtain the corresponding value.